## First step : 
### extracting the data 

In [1]:
!pip install feedparser

import urllib.request
import feedparser

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'all:electron' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 10**4

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url+query).read()

# parse the response using feedparser
feed = feedparser.parse(response)

     |████████████████████████████████| 81kB 5.3MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=c65991c7e3607fb8bc69d7049e0fc1a1ea63bffacd21764e1a4e05b347ef89db
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


### convert it into a list + dataframe 

In [2]:
#coloums of interasts (maybe?)
col=['title', 'summary', 'authors', 'arxiv_primary_category', 'tags']

# Run through each entry, and fill the information into a list
data_list=[]
for c in col:
  abstract_list=[]
  for entry in feed.entries:
    abstract_list.append(entry.get(c))
  data_list.append(abstract_list)

# convert into a panda dataframe (maybe more visible + have some pros I might need)
import pandas as pd
data_df = pd.DataFrame(data_list,index=col)
data_df=data_df.T


In [3]:
data_df.to_csv('arXiv10.csv') 

In [4]:
data_df

,title,summary,authors,arxiv_primary_category,tags
0,Impact of Electron-Electron Cusp on Configurat...,The effect of the electron-electron cusp on th...,"[{'name': 'David Prendergast'}, {'name': 'M. N...","{'term': 'cond-mat.str-el', 'scheme': 'http://...","[{'term': 'cond-mat.str-el', 'scheme': 'http:/..."
1,Electron thermal conductivity owing to collisi...,We calculate the thermal conductivity of elect...,"[{'name': 'P. S. Shternin'}, {'name': 'D. G. Y...","{'term': 'astro-ph', 'scheme': 'http://arxiv.o...","[{'term': 'astro-ph', 'scheme': 'http://arxiv...."
2,Electron pairing: from metastable electron pai...,Starting from the shell structure in atoms and...,"[{'name': 'Guo-Qiang Hai'}, {'name': 'Ladir Câ...","{'term': 'cond-mat.str-el', 'scheme': 'http://...","[{'term': 'cond-mat.str-el', 'scheme': 'http:/..."
3,Electron Temperature Anisotropy and Electron B...,Electron temperature anisotropies and electron...,"[{'name': 'Heyu Sun'}, {'name': 'Jinsong Zhao'...","{'term': 'physics.space-ph', 'scheme': 'http:/...","[{'term': 'physics.space-ph', 'scheme': 'http:..."
4,Hamiltonian of a many-electron system with sin...,Based on the metastable electron-pair energy b...,"[{'name': 'G. -Q. Hai'}, {'name': 'F. M. Peete...","{'term': 'cond-mat.supr-con', 'scheme': 'http:...","[{'term': 'cond-mat.supr-con', 'scheme': 'http..."
...,...,...,...,...,...
1170,Level-resolved quantum statistical theory of e...,The strong mixing of many-electron basis state...,"[{'name': 'J. C. Berengut'}, {'name': 'C. Hara...","{'term': 'physics.atom-ph', 'scheme': 'http://...","[{'term': 'physics.atom-ph', 'scheme': 'http:/..."
1171,Nonlinear aspects of quantum plasma physics,Dense quantum plasmas are ubiquitous in planet...,"[{'name': 'Padma K. Shukla'}, {'name': 'Bengt ...","{'term': 'physics.plasm-ph', 'scheme': 'http:/...","[{'term': 'physics.plasm-ph', 'scheme': 'http:..."
1172,Collisional relaxation of electrons in a warm ...,Extending previous studies of nonthermal elect...,"[{'name': 'E. P. Kontar'}, {'name': 'N. L. S. ...","{'term': 'astro-ph.SR', 'scheme': 'http://arxi...","[{'term': 'astro-ph.SR', 'scheme': 'http://arx..."
1173,Ferromagnetic order of nuclear spins coupled t...,We analyze the ordered state of nuclear spins ...,"[{'name': 'Robert Andrzej Zak'}, {'name': 'Dmi...","{'term': 'cond-mat.mes-hall', 'scheme': 'http:...","[{'term': 'cond-mat.mes-hall', 'scheme': 'http..."


### tokenizing:

In [6]:
!python -m spacy download en_core_web_lg
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=0c5fd3e2294ec04b6fa6dd491ebfc80fe001493d962949be1f237035fb5a6494
  Stored in directory: /tmp/pip-ephem-wheel-cache-jwc5q3lk/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [24]:
import numpy as np

#taking just the titles (data_list[0]). (maybe to use the summary instaed?)
# using lower case. removing extra spaces and '\n ' 
doc=[nlp.tokenizer(text.lower().replace('\n ','').strip()) for text in data_list[0]]

for example:

In [25]:
II=5;
print([token.rank for token in doc[II]])
print([token.lex_id for token in doc[II]])
print([token.text for token in doc[II]])
print([nlp.vocab.strings[token.text] for token in doc[II]])

[15686, 33, 15686, 358821, 29900, 7, 3, 23757, 8, 15686, 18254, 62174, 13, 4085, 17115]
[15686, 33, 15686, 358821, 29900, 7, 3, 23757, 8, 15686, 18254, 62174, 13, 4085, 17115]
['electron', '-', 'electron', 'bremsstrahlung', 'emission', 'and', 'the', 'inference', 'of', 'electron', 'flux', 'spectra', 'in', 'solar', 'flares']
[14911849430818137050, 9153284864653046197, 14911849430818137050, 5074391312506727947, 7093072741639743635, 2283656566040971221, 7425985699627899538, 10770840912364104747, 886050111519832510, 14911849430818137050, 7386502683099369518, 721847661104588153, 3002984154512732771, 3825196732376443040, 54522159426843760]


In [26]:
# Words_vec= words in the text 
# Vocab = unique words in the text

#import numpy as np

#Words=[]
#Words_id=[]
#Max_Length=30;
#Words_vec= np.empty([max_results,Max_Length],dtype='int')


#for i in range(99):
#  #len([token.lex_id for token in doc[i]])
#  Words_vec[i,:len([token.lex_id for token in doc[i]])]=[token.lex_id for token in doc[i]]
#  for token in doc[i]:
#    #print(token.text, token.has_vector, token.vector, token.vector_norm ,"\n")
#    #print(token.text, token.has_vector, token.lex_id ,"\n")
#    Words.append(token.text.lower())
#    Words_id.append(token.lex_id)

    

#Vocab = []
#Vocab_id = []
#[Vocab.append(x) for x in Words if x not in Vocab]
#[Vocab_id.append(id) for id in Words_id if id not in Vocab_id]

#print('number of unique words in our data=',len(Vocab))

from collections import defaultdict

class Vocabulary:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}#{PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 0
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            # First entry of word into vocabulary
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            # Word exists; increase word count
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in [token.text for token in sentence]:
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            # This is the longest sentence
            self.longest_sentence = sentence_len
        # Count the number of sentences
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

voc=Vocabulary('test')
for sent in doc:
  voc.add_sentence(sent)

print('Token 2 corresponds to token:', voc.to_word(2))
print('Token "the" corresponds to index:', voc.to_index('the'))


Token 2 corresponds to token: electron
Token "the" corresponds to index: 31


In [27]:
Input_list=[]
for sample in range(len(doc)):
#  Input_list.append([token.rank for token in doc[sample]])
   Input_list.append([voc.to_index(token.text) for token in doc[sample]])
Output_list=Input_list;
Input_Output_Data_list=[Input_list,Output_list]

In [28]:
print(Input_list[6],doc[6])

[51, 52, 1, 53] improved scenario of baryogenesis


In [29]:
## next: to understand / complete
from sklearn.model_selection import train_test_split

#10% test set
In_train, In_test, Out_train, Out_test = train_test_split(Input_list, Output_list, test_size=0.1, random_state=1)

#from 90% train set --> 20% validation and 80 % training (= in total we have 10% test, 18% val, 72% train )
In_train, In_val, Out_train, Out_val = train_test_split(In_train, Out_train , test_size=0.2, random_state=1)

train_list=  In_train
label=Out_train
validation_list=In_val
test_list=In_test

In [30]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []
    for _sample in batch:
        label_list.append(torch.tensor(_sample))
        text_list.append(torch.tensor(_sample))
    return pad_sequence(label_list, padding_value=0.0), pad_sequence(text_list, padding_value=0.0)

batch_size = 30

def create_iterators(batch_size=batch_size):
    """Heler function to create the iterators"""
    dataloaders = []
    for split in [train_list, validation_list, test_list]:
        dataloader = DataLoader(
            split, batch_size=batch_size,
            collate_fn=collate_batch
            )
        dataloaders.append(dataloader)
    return dataloaders

train_iterator, valid_iterator, test_iterator = create_iterators()

next(iter(train_iterator))




(tensor([[2249,   31,  401,    2,  452,  331,  338,    2,    2,    2,  955,  885,
           741,  526,    2, 2244,  922,  331,  119, 1196,    2,   84,    2,  221,
          1716,   35,  433,    2,   71,  553],
         [ 222,  331,  919,    3,    3,    1,  435,   71,   21,    3,  103,    1,
           742,    3,    3,   25,    1,    1,    2,   16,  219,  596,    3,    3,
             1,  247,  331,    3,  243,   31],
         [ 499,    1,   30,  123, 1951,  595,   88,    3, 1008,    2,   30,   71,
           430,  630,  123, 1929,   35,   31,   97,    5,    3,   96,    2, 1609,
            31,    3,   30,   96,    1,  820],
         [2250,    2, 2224,   25,   64,    5,    1,  178,   30,  137,  240,   74,
             1, 1021,    7, 1256,  240,   74,   30, 1197,  284,   17,   61,   77,
            74, 1618,   35,  746,   16,  885],
         [  46,  321,   25,    2,  187,   71,    2,   94,  131,   25,  501,  664,
             2,    1,   83,   25,    2,    7,  501,  475,  285,   30,   30

In [31]:
next(iter(valid_iterator))


(tensor([[ 741,   31,    2,   64,    2,  467,  898,    2, 1327,   52, 1693,   55,
           320,   64,  468,  866,  393,  957, 1221,  243,   85, 2068,   35,    2,
           667,  773,  820,   82,   71,  426],
         [ 742, 1053,   84,  105,  243,    1,    3,    3,  243,    1,  188,    3,
           321,  103,  630,  747,   71,   16,    2,   78,    2, 2069,   64,    3,
          1887,  205, 1452,    3,  143, 1135],
         [ 152,    1,  747,  234,   78, 1289,    2,   84,    1,   53, 1694,  347,
           322,    1,    3,   19,  482,   18,  141,    1,   61,    3, 1062,    2,
             1,  774,  871,   83,    1,    1],
         [   1,   59,   30,  125,   30,  612,  545,    7,  846,    0,  199,    1,
           323,    2,  658,  105,  283,   10,   19,   35,   30,  657,  137,    7,
            31,    2,    1, 1159,   16,   41],
         [ 579,  130, 1970,   31,  221, 1382,  899,  392,   16,    0,   18,   41,
           324,    3,  557,  867,  153,   25,   41, 1189,  439,  137, 1292

In [32]:
print(Input_list[6],doc[6])

[51, 52, 1, 53] improved scenario of baryogenesis


In [33]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [34]:
ntokens = (voc.num_words)#max(Vocab_id) # the size of vocabulary
device='cpu'
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
bptt=35
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [35]:
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

train_data=In_train

def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    for batch, i in enumerate(range(0, len(train_data) - 1, bptt)):
        train_iterator, valid_iterator, test_iterator = create_iterators()
        data, targets=next(iter(train_iterator))
        optimizer.zero_grad()
        if data.size(0) != bptt:
            src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            train_iterator, valid_iterator, test_iterator = create_iterators()
            data, targets=next(iter(valid_iterator))
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = eval_model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output_flat, targets.view(-1)).item()
    return total_loss / (len(data_source) - 1)

In [36]:
best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

val_data, targets =next(iter(valid_iterator))

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  2.09s | valid loss  2.50 | valid ppl    12.16
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  2.07s | valid loss  2.53 | valid ppl    12.52
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  2.07s | valid loss  2.54 | valid ppl    12.71
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   4 | time:  2.06s | valid loss  2.55 | valid ppl    12.84
--------------------------------------------------------------------------

##Generating...

In [37]:
next(iter(train_iterator))

(tensor([[2249,   31,  401,    2,  452,  331,  338,    2,    2,    2,  955,  885,
           741,  526,    2, 2244,  922,  331,  119, 1196,    2,   84,    2,  221,
          1716,   35,  433,    2,   71,  553],
         [ 222,  331,  919,    3,    3,    1,  435,   71,   21,    3,  103,    1,
           742,    3,    3,   25,    1,    1,    2,   16,  219,  596,    3,    3,
             1,  247,  331,    3,  243,   31],
         [ 499,    1,   30,  123, 1951,  595,   88,    3, 1008,    2,   30,   71,
           430,  630,  123, 1929,   35,   31,   97,    5,    3,   96,    2, 1609,
            31,    3,   30,   96,    1,  820],
         [2250,    2, 2224,   25,   64,    5,    1,  178,   30,  137,  240,   74,
             1, 1021,    7, 1256,  240,   74,   30, 1197,  284,   17,   61,   77,
            74, 1618,   35,  746,   16,  885],
         [  46,  321,   25,    2,  187,   71,    2,   94,  131,   25,  501,  664,
             2,    1,   83,   25,    2,    7,  501,  475,  285,   30,   30

In [66]:
#test1= torch.as_tensor([4,5,7,1,81])

def generate(model, data_source):
    model.eval() # Turn on the evaluation mode
    total_loss = 0.
    src_mask = model.generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            train_iterator, valid_iterator, test_iterator = create_iterators()
            data, targets=next(iter(valid_iterator))
            if data.size(0) != bptt:
                src_mask = model.generate_square_subsequent_mask(data.size(0)).to(device)
            output = model(data, src_mask)
    return output

#test11, targets=next(iter(test_iterator))
test1=torch.as_tensor(Input_list[2])#.tolist()
#print(torch.argmax(generate(model,test1),dim=2))
out=torch.argmax(generate(model,test1),dim=2)
#print(out.shape,test1.shape)
print([voc.to_word(index) for index in out[:,20].tolist()])

#src_mask=model.generate_square_subsequent_mask(test1.size(0)).to(device)
#print(model.forward(test1,src_mask).shape)
#print(torch.argmax(model.forward(test1,src_mask),dim=2))
#out=torch.argmax(model.forward(test1,src_mask),dim=2)
#print([voc.to_word(index) for index in test1.tolist()])
#print([voc.to_word(index) for index in out[4].tolist()])


['disordered', 'electron', 'interactions', 'in', 'metal', 'films', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact', 'impact']


In [1]:
print([voc.to_word(index) for index in test1.tolist()])

NameError: ignored